In [2]:
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Custom layer definition
class CustomScaleLayer(Layer):
    def __init__(self, scale_factor=1.0, **kwargs):
        super(CustomScaleLayer, self).__init__(**kwargs)
        self.scale_factor = scale_factor

    def call(self, inputs):
        return inputs * self.scale_factor

    def get_config(self):
        config = super(CustomScaleLayer, self).get_config()
        config.update({'scale_factor': self.scale_factor})
        return config

# Function to load data
dataset_path = 'test'

def load_data(dataset_path):
    images = []
    labels = []
    class_names = os.listdir(dataset_path)
    for idx, class_name in enumerate(class_names):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path): # Skip non-directory entries
            continue
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
                    image = tf.keras.preprocessing.image.img_to_array(image)
                    images.append(image)
                    labels.append(idx)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

images, labels, class_names = load_data(dataset_path)

# Check if images and labels were loaded
print(f"Number of images loaded: {len(images)}")
print(f"Number of labels loaded: {len(labels)}")

# Splitting into train and test data
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create a mapping from class indices to class names
class_index_to_name = {idx: class_name for idx, class_name in enumerate(class_names)}

# Print the mapping
print("Class Index to Name Mapping:")
for idx, class_name in class_index_to_name.items():
    print(f"{idx}: {class_name}")

# Data generators with enhanced data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Reducing the batch size
train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=32)

# Load the InceptionResNetV2 model without the top layers
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Fine-tune more layers
for layer in base_model.layers[:250]:
    layer.trainable = False

for layer in base_model.layers[250:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = CustomScaleLayer(scale_factor=0.5)(x)  # Custom layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(class_names), activation='softmax')(x)  # Adjust the number of classes as needed

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with a reduced learning rate and mixed precision loss scaling
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
]

# Train the model with increased epochs
history = model.fit(train_generator, validation_data=test_generator, epochs=100, callbacks=callbacks)

# Save the model to Google Drive
model.save('final_model.h5', save_format='h5')

# Save the training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Load the model with custom objects
model_path = 'final_model.h5'
custom_objects = {'CustomScaleLayer': CustomScaleLayer}
model = load_model(model_path, custom_objects=custom_objects)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Generate a summary report
def plot_training_history(history):
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    # Plot accuracy
    axs[0].plot(history['accuracy'], label='Training Accuracy')
    axs[0].plot(history['val_accuracy'], label='Validation Accuracy')
    axs[0].set_title('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend(loc='lower right')

    # Plot loss
    axs[1].plot(history['loss'], label='Training Loss')
    axs[1].plot(history['val_loss'], label='Validation Loss')
    axs[1].set_title('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Loss')
    axs[1].legend(loc='upper right')

    plt.tight_layout()
    plt.show()

plot_training_history(history.history)

# Function to load and preprocess the image
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))  # Load the image and resize
    image = tf.keras.preprocessing.image.img_to_array(image)  # Convert to array
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image / 255.0  # Rescale
    return image

# Function to predict the class of the image
def predict_image(image_path, model, class_names):
    image = preprocess_image(image_path)
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the index of the highest probability
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name

# Path to the image you want to predict
image_path = 'test/Contact Dermatitis/acute-paronychia-2.jpg'  # Replace with the actual path to your image

# Predict the class
predicted_class_name = predict_image(image_path, model, class_names)

# Print the result
print(f'The predicted class for the image is: {predicted_class_name}')


Number of images loaded: 1862
Number of labels loaded: 1862
Class Index to Name Mapping:
0: Atopic Dermatitis
1: Contact Dermatitis
2: Dyshidrotic Eczema
3: Nummular Dermatitis
4: Seborrheic Dermatitis
5: Stasis Dermatitis


KeyboardInterrupt: 

In [1]:
import io
import numpy as np
from flask import Flask, request, jsonify
from PIL import Image
from werkzeug.utils import secure_filename
import tensorflow as tf
from tensorflow.keras.models import load_model
from flask_cors import CORS

# Assuming you have the model loaded and class names ready
class CustomScaleLayer(tf.keras.layers.Layer):
    def __init__(self, scale_factor=1.0, **kwargs):
        super(CustomScaleLayer, self).__init__(**kwargs)
        self.scale_factor = scale_factor

    def call(self, inputs):
        return inputs * self.scale_factor

    def get_config(self):
        config = super(CustomScaleLayer, self).get_config()
        config.update({'scale_factor': self.scale_factor})
        return config

# Load your model and class names
model = load_model('final_model.h5', custom_objects={'CustomScaleLayer': CustomScaleLayer})
class_names = ['Atopic Dermatitis', 'Contact Dermatitis', 'Dyshidrotic Eczema', 'Nummular Dermatitis', 'Seborrheic Dermatitis', 'Stasis Dermatitis']

app = Flask(__name__)
CORS(app)  # Enable CORS

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})
    
    if file:
        filename = secure_filename(file.filename)
        img_bytes = file.read()
        img = Image.open(io.BytesIO(img_bytes))
        img = img.resize((224, 224))  # Resize to match the model input size
        img = np.array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0  # Rescale
        
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions, axis=1)[0]  # Get the index of the highest probability
        predicted_class_name = class_names[predicted_class]
        
        response = {'predicted_class': predicted_class_name}
        print(response)  # Log the response
        return jsonify(response)

if __name__ == '__main__':
    import threading

    def run_flask():
        app.run(host='0.0.0.0', port=5000)

    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.5:5000
Press CTRL+C to quit


1/1 [==============================] - 7s 7s/step


127.0.0.1 - - [31/Jul/2024 10:58:02] "POST /predict HTTP/1.1" 200 -


{'predicted_class': 'Seborrheic Dermatitis'}
